In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import missingno as msno
from pathlib import Path
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
train=pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
test=pd.read_csv('../input/petfinder-pawpularity-score/test.csv')

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import KFold
import tensorflow as tf

In [ ]:
train["file_path"] = train["Id"].apply(lambda x: "../input/petfinder-pawpularity-score/train/" + x + ".jpg")
test["file_path"] = test["Id"].apply(lambda x: "../input/petfinder-pawpularity-score/test/" + x + ".jpg")

In [ ]:
image_size = 128
def preprocess(image_url):
    image_string = tf.io.read_file(image_url)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.central_crop(image, 1.0)
    image = tf.image.resize(image, (image_size, image_size))
    return image
x_train=[]
for i in train['file_path']:
    x1=preprocess(i)
    x_train.append(x1)

In [ ]:
test2=[]
for i in test['file_path']:
    x1=preprocess(i)
    test2.append(x1)
test2=np.array(test2)

In [ ]:
x_train=np.array(x_train)
y_train=train['Pawpularity']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_train,y_train,test_size=0.2)

In [ ]:
inputs=keras.Input(shape=(128,128,3))
x=inputs
x=keras.layers.Conv2D(filters=8,kernel_size=3,strides=2,padding='same',activation='relu')(x)
# x=keras.layers.BatchNormalization()(x)
# x=keras.layers.Dropout(0.2)(x)
# x=keras.layers.MaxPool2D(pool_size=2)(x)
# x=keras.layers.BatchNormalization()(x)
# x=keras.layers.Dropout(0.2)(x)
x=keras.layers.Conv2D(filters=16,kernel_size=3,strides=2,padding='same',activation='relu')(x)
# x=keras.layers.BatchNormalization()(x)
# x=keras.layers.MaxPool2D(pool_size=2)(x)
# x=keras.layers.BatchNormalization()(x)
x=keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding='same',activation='relu')(x)
x=keras.layers.Flatten()(x)
x=keras.layers.Dense(128, activation = "relu")(x)
x=keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(inputs=inputs, outputs=output)

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(
    patience=5
    )
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    factor=0.1,
    patience=2, 
    min_lr=1e-9
    )
callbacks = [early_stop,reduce_lr]

In [ ]:
model.compile(loss='mse', optimizer='Adam', metrics=[tf.keras.metrics.RootMeanSquaredError(name="rmse"), "mae", "mape"])

In [ ]:
history = model.fit(x_train,y_train, epochs=25,batch_size=64,validation_data = (x_test,y_test),callbacks=callbacks)

In [ ]:
cnn_pred=model.predict(test2)

In [ ]:
cnn=pd.DataFrame()
cnn['Id']=test['Id']
cnn['Pawpularity']=cnn_pred

# **LightGBM**

In [ ]:
cols=['Id','Pawpularity','file_path']
x_train=train.drop(cols,axis=1)

In [ ]:
y_train=train['Pawpularity']

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x_train,y_train,test_size=0.2)

In [ ]:
import lightgbm as lgb

In [ ]:
lgbr=lgb.LGBMRegressor(max_depth=10,learning_rate=0.005,n_estimators=200,reg_alpha=1,reg_lambda=0.1)
lgbr.fit(x_train,y_train)
y_pred=lgbr.predict(x_test)

In [ ]:
from sklearn.metrics import mean_squared_error,r2_score
def rmse(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))
print(rmse(y_test,y_pred))
print(mean_squared_error(y_test,y_pred))
print(r2_score(y_test,y_pred))

In [ ]:
test1=test.drop('Id',axis=1)
test2=test1.iloc[:,:-1]
Pred=lgbr.predict(test2)

In [ ]:
sub=pd.DataFrame()
sub['Id']=test['Id']
sub['Pawpularity']=Pred

In [ ]:
sub.head()

In [ ]:
cnn.head()

In [ ]:
merge=pd.DataFrame()
merge['Pawpularity']=cnn['Pawpularity']/2+sub['Pawpularity']/2

In [ ]:
merge['Id']=sub['Id']

In [ ]:
merge.to_csv('submission.csv',index=False)

# Inference
I got 20.47 rmse

It's better than single model

wow...I will it them and look for a better score

**Could you give me some advices...?**